In [1]:
#!/usr/bin/env python

# Copyright 2016 NeuroData (http://neurodata.io)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# qa_tensor.py
# Created by Vikram Chandrashekhar.
# Edited by Greg Kiar.
# Email: Greg Kiar @ gkiar@jhu.edu

from dipy.reconst.dti import fractional_anisotropy, color_fa
from argparse import ArgumentParser
from scipy import ndimage
import os
import re
import numpy as np
import nibabel as nb
import sys
import matplotlib

matplotlib.use('Agg')  # very important above pyplot import
import matplotlib.pyplot as plt


def tensor2fa(tensors, tensor_name, dti, derivdir, qcdir):
    '''
    outdir: location of output directory.
    fname: name of output fa map file. default is none (name created based on
    input file)
    '''
    dti_data = nb.load(dti)
    affine = dti_data.get_affine()
    dti_data = dti_data.get_data()

    # create FA map
    FA = fractional_anisotropy(tensors.evals)
    FA[np.isnan(FA)] = 0

    # generate the RGB FA map
    FA = np.clip(FA, 0, 1)
    RGB = color_fa(FA, tensors.evecs)

    fname = os.path.split(tensor_name)[1].split(".")[0] + '_fa_rgb.nii.gz'
    fa = nb.Nifti1Image(np.array(255 * RGB, 'uint8'), affine)
    nb.save(fa, derivdir + fname)

    fa_pngs(fa, fname, qcdir)


def fa_pngs(data, fname, outdir):
    '''
    data: fa map
    '''
    im = data.get_data()
    fig = plot_rgb(im)
    fname = os.path.split(fname)[1].split(".")[0] + '.png'
    plt.savefig(outdir + fname, format='png')


def plot_rgb(im):
    plt.rcParams.update({'axes.labelsize': 'x-large',
                         'axes.titlesize': 'x-large'})

    if im.shape == (182, 218, 182):
        x = [78, 90, 100]
        y = [82, 107, 142]
        z = [88, 103, 107]
    else:
        shap = im.shape
        x = [int(shap[0]*0.35), int(shap[0]*0.51), int(shap[0]*0.65)]
        y = [int(shap[1]*0.35), int(shap[1]*0.51), int(shap[1]*0.65)]
        z = [int(shap[2]*0.35), int(shap[2]*0.51), int(shap[2]*0.65)]
    coords = (x, y, z)

    labs = ['Sagittal Slice (YZ fixed)',
            'Coronal Slice (XZ fixed)',
            'Axial Slice (XY fixed)']
    var = ['X', 'Y', 'Z']

    idx = 0
    for i, coord in enumerate(coords):
        for pos in coord:
            idx += 1
            ax = plt.subplot(3, 3, idx)
            ax.set_title(var[i] + " = " + str(pos))
            if i == 0:
                image = ndimage.rotate(im[pos, :, :], 90)
            elif i == 1:
                image = ndimage.rotate(im[:, pos, :], 90)
            else:
                image = im[:, :, pos]

            if idx % 3 == 1:
                ax.set_ylabel(labs[i])
                ax.yaxis.set_ticks([0, image.shape[0]/2, image.shape[0] - 1])
                ax.xaxis.set_ticks([0, image.shape[1]/2, image.shape[1] - 1])

            plt.imshow(image)

    fig = plt.gcf()
    fig.set_size_inches(12.5, 10.5, forward=True)
    return fig

In [2]:
from dipy.reconst.dti import fractional_anisotropy, color_fa
from argparse import ArgumentParser
from scipy import ndimage
import os
import re
import numpy as np
import nibabel as nb
import sys
import matplotlib

matplotlib.use('Agg')  # very important above pyplot import
import matplotlib.pyplot as plt

In [3]:
img = nb.load('../../../../../Desktop/result/dogsig1_gausig2.3/v100_ch0_tensorfsl_dogsig1_gausig2.3.nii')

In [4]:
img.shape

(5000, 5000, 5, 6)

In [5]:
data = img.get_data()

In [6]:
from dipy.segment.mask import median_otsu

maskdata, mask = median_otsu(data, 3, 1)
print('maskdata.shape (%d, %d, %d, %d)' % maskdata.shape)

/Users/Tony/anaconda/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


maskdata.shape (5000, 5000, 5, 6)


In [7]:
import dipy.reconst.dti as dti

In [8]:
img3 = nb.load('../../../../../Desktop/result/v100_ch0_gv_dogsig1.nii')

In [9]:
img3.shape

(5000, 5000, 5, 3)

In [17]:
data3 = img3.get_data()

In [45]:
data3_slice1 = data3[:, :, 0, 0]

In [48]:
print data3_slice1.shape

(5000, 5000)


In [51]:
data3_slice1_eigvec = data3[:, :, 0, :]
print data3_slice1_eigvec.shape

(5000, 5000, 3)


In [52]:
FA = fractional_anisotropy(data3_slice1_eigvec)
##????

In [53]:
FA[np.isnan(FA)] = 0

In [54]:
print FA.shape

(5000, 5000)


In [33]:
abc = np.ones((5000, 5000))

In [55]:
FA.shape

(5000, 5000)

In [56]:
v.shape[-2:]

(5000, 5000)

In [58]:
if (FA.shape != v.shape) or ((3, 3) != v.shape[-2:]):
    print 'help'
    if (FA.shape != v.shape):
        print 'yes'
    if ((3,3) != v.shape[-2:]):
        print 'oh no'

help
oh no


In [59]:
RGB = color_fa(FA, v) 

ValueError: Wrong number of dimensions for evecs

In [43]:
from numpy import linalg as LA

In [49]:
w, v = LA.eigh(data3_slice1)

In [50]:
print v.shape

(5000, 5000)


In [67]:
print w.shape

(5000,)


In [ ]:
img2 = nb.load('../../../../../Desktop/result/dogsig1_gausig2.3/v100_ch0_tensordtk_dogsig1_gausig2.3_tensor.nii')
img2.shape

In [68]:
from_lower_triangular(img)

NameError: name 'from_lower_triangular' is not defined